#### Author: Jamilson Junior
#### Sources/Credits:
- [categorical](https://www.kaggle.com/danaugrs/categorical-column-types)
- [numeric](https://www.kaggle.com/aakashveera/random-forest)
- [time](https://www.kaggle.com/gaborfodor/notebookd19d11e4f2)

# Bosch Production Line Performance Solution - Data Exploration

## Problem Description
The objective of this challenge is to predict which parts will fail quality control along the assembly line of a Bosch manufacturing facility. In this case, there are 3 lines, with 52 stations each.

## Brief Data Description
This dataset represents measurements of parts as they move through a production line. The **objective is to predict which parts will fail quality control**.

This dataset has a large number of anonymized features. The features are named according to the following convention:

L3_S36_F3939 &#8594; **Line** 3, **Station** 36 and **Feature** 3939.

Given the large size of the dataset, the data is separated into different files, according to its type:
- Numerical
- Categorical
- Date

Note: The ***Date*** data has the following naming convention:

L0_S0_D1 &#8594; **Time stamp** of the measurement taken in the **Line** 0, **Station** 0 of the **Feature** 0.

The **objective** is to predict if the part will fail quality control, represented by the feature ***Response = 1***.

## Data Exploration

In [ ]:
from pathlib import Path
import os
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot as plt
from collections import Counter, defaultdict
import warnings
import zipfile
warnings.filterwarnings("ignore")
print('Imports: Done')

### Numeric Values Exploratory Analysis

In [ ]:
print(os.listdir('..'))
with zipfile.ZipFile("../input/bosch-production-line-performance/train_numeric.csv.zip","r") as z:
    z.extractall("..")

In [ ]:
os.listdir('..')

In [ ]:
train_numeric_chunks = pd.read_csv(Path().joinpath('../train_numeric.csv'), chunksize
=50000)
print('Numeric data import: Done')

In [ ]:
train_numeric_chunks # Text file reader

In [ ]:
# For numeric Feature selection
model = xgb.XGBClassifier(n_estimators=30)

In [ ]:
ctr=0
for chunk in train_numeric_chunks:
    model.fit(chunk.drop(['Id','Response'],axis=1),chunk['Response'])
    ctr+=1
    print(ctr)
print('Number of chunks: ', ctr)

In [ ]:
fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(model,ax=ax, max_num_features=15)

In [ ]:
imp = model.feature_importances_
name =model.get_booster().feature_names
most_imp_feat = pd.DataFrame({'feat':name,'impo':imp}).sort_values(by='impo',ascending=False)
most_imp_feat.head(15)

In [ ]:
chosen_feats = most_imp_feat['feat'].to_list()[:15]
print(chosen_feats)

### Categorical Values Exploratory Analysis

In [ ]:
print(os.listdir('..'))
with zipfile.ZipFile("../input/bosch-production-line-performance/train_categorical.csv.zip","r") as z:
    z.extractall("..")
with zipfile.ZipFile("../input/bosch-production-line-performance/test_categorical.csv.zip","r") as z:
    z.extractall("..")

In [ ]:
train_quick_view = pd.read_csv(Path().joinpath('../train_categorical.csv'), nrows=1000)
train_quick_view

Analysing a small part of the training dataset we can see that we have a very sparse dataset.

Let us understand better what kind of data we have in this dataset.

In [ ]:
train_categorical_chunks = pd.read_csv(Path().joinpath('../train_categorical.csv'),dtype=str, usecols=list(range(1,2141)), chunksize=100000)
test_categorical_chunks = pd.read_csv(Path().joinpath('../test_categorical.csv'),dtype=str, usecols=list(range(1,2141)), chunksize=100000)

In [ ]:
trainingCounts = defaultdict(Counter)
for idx, chunk in enumerate(train_categorical_chunks):
    for col in chunk:
        trainingCounts[col] += Counter(chunk[col].values)
    print('Done with chunk {0}/12'.format(idx+1))
testCounts = defaultdict(Counter)
for idx, chunk in enumerate(test_categorical_chunks):
    for col in chunk:
        testCounts[col] += Counter(chunk[col].values)
    print('Done with chunk {0}/12'.format(idx+1))

In [ ]:
emptyColumnsTest = [col for col in testCounts if len(list(testCounts[col].keys())) == 1]
binaryColumnsTest = [col for col in testCounts if len(list(testCounts[col].keys())) == 2]
multiColumnsTest = [col for col in testCounts if len(list(testCounts[col].keys())) > 2]

emptyColumnsTrain = [col for col in trainingCounts if len(list(trainingCounts[col].keys())) == 1]
binaryColumnsTrain = [col for col in trainingCounts if len(list(trainingCounts[col].keys())) == 2]
multiColumnsTrain = [col for col in trainingCounts if len(list(trainingCounts[col].keys())) > 2]

print('{0:5} | {1:^5} | {2:^7} | {3:^5}'.format('', 'Empty', 'Binary', 'Multi'))
print('{0:5} | {1:^5} | {2:^7} | {3:^5}'.format('Test', len(emptyColumnsTest), len(binaryColumnsTest), len(multiColumnsTest)))
print('{0:5} | {1:^5} | {2:^7} | {3:^5}'.format('Train', len(emptyColumnsTrain), len(binaryColumnsTrain), len(multiColumnsTrain)))

In [ ]:
import seaborn as sns
%matplotlib inline

sns.set_style("whitegrid")
sns.set_context("talk")
sns.barplot(y='Value', x='Type', hue='Dataset', data=pd.DataFrame([
    ['Test', 'Empty', len(emptyColumnsTest)],
    ['Test', 'Binary', len(binaryColumnsTest)],
    ['Test', 'Multi-value', len(multiColumnsTest)],
    ['Train', 'Empty', len(emptyColumnsTrain)],
    ['Train', 'Binary', len(binaryColumnsTrain)],
    ['Train', 'Multi-value', len(multiColumnsTrain)]],
    columns=['Dataset', 'Type', 'Value'])).set(xlabel='', ylabel='Value')

As we can see, the number of columns does not match between test and train data.

In [ ]:
sns.set_context("talk")
sns.heatmap(pd.DataFrame([
    [len(set(emptyColumnsTest).intersection(set(emptyColumnsTrain))),
     len(set(emptyColumnsTest).intersection(set(binaryColumnsTrain))),
     len(set(emptyColumnsTest).intersection(set(multiColumnsTrain)))],
    [len(set(binaryColumnsTest).intersection(set(emptyColumnsTrain))),
     len(set(binaryColumnsTest).intersection(set(binaryColumnsTrain))),
     len(set(binaryColumnsTest).intersection(set(multiColumnsTrain)))],
    [len(set(multiColumnsTest).intersection(set(emptyColumnsTrain))),
     len(set(multiColumnsTest).intersection(set(binaryColumnsTrain))),
     len(set(multiColumnsTest).intersection(set(multiColumnsTrain)))]],
    columns=['Train Empty', 'Train Binary', 'Train Multi-value'],
    index=['Test Empty', 'Test Binary', 'Test Multi-value']),
            annot=True, fmt="d", linewidths=.5)

In [ ]:
trulyEmpty = set(emptyColumnsTest).intersection(set(emptyColumnsTrain))
trulyMulti = set(multiColumnsTest).union(set(multiColumnsTrain))
trulyBinary = set(binaryColumnsTest).union(set(binaryColumnsTrain)) - trulyMulti

len(trulyEmpty) + len(trulyBinary) + len(trulyMulti)

From the heatmap we can see that 56 columns are truly empty across training and test data, thus these can be safely ignored.

![alt text](categorical_analysis.PNG)

### Date Values Exploratory Analysis

In [ ]:
print(os.listdir('..'))
with zipfile.ZipFile("../input/bosch-production-line-performance/train_date.csv.zip","r") as z:
    z.extractall("..")
with zipfile.ZipFile("../input/bosch-production-line-performance/test_date.csv.zip","r") as z:
    z.extractall("..")

In [ ]:
train_date_part = pd.read_csv(Path().joinpath('../train_date.csv'), nrows=10000)
print(train_date_part.shape)
print('Percentage of missing data {}'.format(train_date_part.count().sum() / train_date_part.size * 100))
train_date_part.head(2)

In [ ]:
def get_station_times(dates, withId=False):
    times = []
    cols = list(dates.columns)
    if 'Id' in cols:
        cols.remove('Id')
    for feature_name in cols:
        if withId:
            df = dates[['Id', feature_name]].copy()
            df.columns = ['Id', 'time']
        else:
            df = dates[[feature_name]].copy()
            df.columns = ['time']
        df['station'] = feature_name.split('_')[1][1:]
        df = df.dropna()
        times.append(df)
    return pd.concat(times)

station_times = get_station_times(train_date_part, withId=True).sort_values(by=['Id', 'station'])
min_station_times = station_times.groupby(['Id', 'station']).min()['time'] # Data frame with min times by Id/Station
max_station_times = station_times.groupby(['Id', 'station']).max()['time'] # Data frame with max times by Id/Station

In [ ]:
max_station_times.head(5)

In [ ]:
print('The percentage of Id/Station pairs with same min/max value is: \n {}'.format(np.mean(1. * (min_station_times == max_station_times))))

After removing missing values, we can observe that the times observations are almost always unique for each Id/Station pair, we can save a lot of memory by just reading the first value.



In [ ]:
date_cols = train_date_part.drop('Id', axis=1).count().reset_index().sort_values(by=0, ascending=False)
date_cols['station'] = date_cols['index'].apply(lambda s: s.split('_')[1])
date_cols = date_cols.drop_duplicates('station', keep='first')['index'].tolist()
print(len(date_cols)) # 52 stations -> Selected stations

In [ ]:
train_date = pd.read_csv(Path().joinpath('../train_date.csv'), usecols=date_cols)
test_date = pd.read_csv(Path().joinpath('../test_date.csv'), usecols=date_cols)

In [ ]:
train_date.head(3)

In [ ]:
print('Training date data shape: {}'.format(train_date.shape))
print('Testing date data shape: {}'.format(test_date.shape))

In [ ]:
train_station_times = get_station_times(train_date, withId=False)
test_station_times = get_station_times(test_date, withId=False)

In [ ]:
train_time_cnt = train_station_times.groupby('time').count()[['station']].reset_index()
train_time_cnt.columns = ['time', 'cnt']
test_time_cnt = test_station_times.groupby('time').count()[['station']].reset_index()
test_time_cnt.columns = ['time', 'cnt']

In [ ]:
fig = plt.figure()
plt.plot(train_time_cnt['time'].values, train_time_cnt['cnt'].values, 'b.', alpha=0.01, label='train')
plt.plot(test_time_cnt['time'].values, test_time_cnt['cnt'].values, 'r.', alpha=0.01, label='test')
plt.title('Original date values')
plt.ylabel('Number of records')
plt.xlabel('Time')
plt.show()
print('(min, max) train time:{}'.format((train_time_cnt['time'].min(), train_time_cnt['time'].max())))
print('(min, max) test time:{}'.format((test_time_cnt['time'].min(), test_time_cnt['time'].max())))

Observations:
- Train and test have the same time period.
- Periodic pattern
- Date range from 0-1718, granularity of 0.01.
- Gap at around 800

What does granularity of 0.01 mean?

In [ ]:
time_ticks = np.arange(train_time_cnt['time'].min(), train_time_cnt['time'].max() + 0.01, 0.01)
time_ticks = pd.DataFrame({'time': time_ticks})
time_ticks = pd.merge(time_ticks, train_time_cnt, how='left', on='time')
time_ticks = time_ticks.fillna(0)
# time_ticks

In [ ]:
x = time_ticks['cnt'].values
max_lag = 8000
auto_corr_ks = range(1, max_lag)
auto_corr = np.array([1] + [np.corrcoef(x[:-k], x[k:])[0, 1] for k in auto_corr_ks])
fig = plt.figure()
plt.plot(auto_corr, 'k.', label='autocorrelation by 0.01')
plt.title('Train Sensor Time Auto-correlation')
# Period of 25
period = 25
auto_corr_ks = list(range(period, max_lag, period))
auto_corr = np.array([1] + [np.corrcoef(x[:-k], x[k:])[0, 1] for k in auto_corr_ks])
plt.plot([0] + auto_corr_ks, auto_corr, 'go', alpha=0.5, label='strange autocorrelation at 0.25')
# Period 1675 (week)
period = 1675
auto_corr_ks = list(range(period, max_lag, period))
auto_corr = np.array([1] + [np.corrcoef(x[:-k], x[k:])[0, 1] for k in auto_corr_ks])
plt.plot([0] + auto_corr_ks, auto_corr, 'ro', markersize=10, alpha=0.5, label='one week = 1675?')
plt.xlabel('k * 0.01 -  autocorrelation lag')
plt.ylabel('autocorrelation')
plt.legend(loc=0)
plt.grid()

We can observe  peaks with period of 1675, we can call the a week.

In [ ]:
week_duration = 1679
train_time_cnt['week_part'] = ((train_time_cnt['time'].values * 100) % week_duration).astype(np.int64)

In [ ]:
train_week_part = train_time_cnt.groupby(['week_part'])[['cnt']].sum().reset_index()
fig = plt.figure()
plt.plot(train_week_part.week_part.values, train_week_part.cnt.values, 'b.', alpha=0.5, label='train count')
y_train = train_week_part['cnt'].rolling(window=20, center=True).mean().values
plt.plot(train_week_part.week_part.values, y_train, 'b-', linewidth=4, alpha=0.5, label='train count smooth')
plt.title('Relative Part of week')
plt.ylabel('Number of records')
plt.xlim(0, 1680)
plt.grid()